In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import pandas as pd
import shutil
import time
import csv
import os

In [2]:
def txt_to_csv(filename, filedest, tab_type):
    
    
    if os.path.isfile(filedest + '.csv'):
        os.remove(filedest + '.csv')
        
    file1 = open(filename + tab_type, "rb")
    file_csv = open(filedest + tab_type + '.csv', 'w+', newline ='')

    count = 0
    flag_data = 0

    with file_csv:
        for line in file1:

            if b'\xb0C' in line:     # se trovo il carattere °C lo ignoro
                pass
            else:
                line = line.decode() # altrimenti decodifico la riga

                # cerco la riga con i nomi della tabella
                if line.startswith('NR'):
                    flag_data = 1                  # quando trovo i nomi delle colonne imposto flag_data = 1
                    table_names = line.split()     # divido i nomi delle colonne      
                    table_names.remove('LastLine') # elimino 'LastLine' che fa riferimento ad un singolo dato duplicato
                    #for item in table_names: print(item)
                    write = csv.writer(file_csv)  
                    write.writerow(table_names)
                    #print(table_names)

                # Se sono arrivato alla tabella ma la riga inizia con 0 la salto
                elif flag_data == 1 and line.startswith('0'):
                    pass

                # Se sono arrivato alla tabella (flag_data == 1) e non sono alla prima riga
                # inizio a salvare i dati
                elif flag_data == 1:
                    row = line.split()

                    if not line.startswith('[END]') and len(row) == len(table_names)+1: # le righe con meno di 25 elementi hanno dati mancanti
                        row[-2] = str(row[-2]) + ' ' + str(row[-1]) # unisco l'ora col giorno
                        del row[-1] # elimino la colonna con i giorni
                        write = csv.writer(file_csv)
                        write.writerow(row)
                        count = count+1      # aggiorno il contatore delle righe

### Coversione da .txt/.tab a Dataframe
* creo un DataFrame vuoto
* quando raggiungo la riga dei nomi delle colonne riporto gli stessi nomi per le colonne del DataFrame
* aggiungo ogni riga trovata al dataframe

In [3]:
#--- FUNZIONE PER CONVERTIRE UNA TABELLA IN DATAFRAME

def tab_to_df(filename):
        
    file1 = open(filename, "rb")
    df = pd.DataFrame()
    flag_data = 0
    
    pbar = tqdm(file1, desc=filename)
    for line in pbar:

        if b'\xb0C' in line:     # se trovo il carattere °C lo ignoro
            pass
        else:
            line = line.decode() # altrimenti decodifico la riga

            # cerco la riga con i nomi della tabella
            if line.startswith('NR'):
                flag_data = 1                  # quando trovo i nomi delle colonne imposto flag_data = 1
                table_names = line.split()     # divido i nomi delle colonne      
                table_names.remove('LastLine') # elimino 'LastLine' che fa riferimento ad un singolo dato duplicato
                
                # definisco i nomi delle colonne del dataframe
                df = df.reindex(columns = table_names)

            # Se sono arrivato alla tabella ma la riga inizia con 0 la salto
            elif flag_data == 1 and line.startswith('0'):
                pass

            # Se sono arrivato alla tabella (flag_data == 1) e non sono alla prima riga
            # inizio a salvare i dati
            elif flag_data == 1:
                row = line.split()

                if not line.startswith('[END]') and len(row) == len(table_names)+1: # le righe con meno di 25 elementi hanno dati mancanti
                    row[-2] = str(row[-2]) + ' ' + str(row[-1]) # unisco l'ora col giorno
                    del row[-1] # elimino la colonna con i giorni
                    
                    df_length = len(df)
                    df.loc[df_length] = row
    
    df.set_index('DataTime', drop = True, inplace=True)
    df.index = pd.to_datetime(df.index)
    df.drop('NR', axis=1, inplace=True)
    return df
                        
#--- MAIN

#data = tab_to_df('table_new/Diag_temp.tab')
#data.head()

In [4]:
def load_data(file):
    #column_subset = ['t_Mach',   't_HydrTa', 't_TTlubr', 't_TT_ab1', 
                     #'t_TT_ab2', 't_HdStUp', 't_HdStLw', 't_Sfrnt1', 
                     #'t_Sfrnt2', 't_Srear',  't_Smotor', 'DataTime']

    df = pd.read_csv (file,
                      #usecols=column_subset,
                      index_col=False)

    df['DataTime'] = pd.to_datetime(df['DataTime'], format='%H:%M:%S %d.%m.%Y')
    
    df.set_index('DataTime', drop = True, inplace=True)
    
    if not df.index.is_monotonic:
        df.sort_index(inplace = True)
    
    df.drop('NR', axis=1, inplace=True)
    
    return df

In [56]:
path = 'table_new/'
path_dest = path + 'csv/'
tab_name = 'Diag_temp'
df_list = []

# controllo se la cartella 'csv' è presente. Se c'è la elimino e la ricreo, se non c'è la creo.
if os.path.isdir(path_dest):
    shutil.rmtree(path_dest)
    os.mkdir(path_dest)
else:
    os.mkdir(path_dest)

for filename in os.listdir(path):
    if tab_name in filename and (filename.endswith('.txt') or filename.endswith('.tab')): 
        df = tab_to_df(path + filename)
        df_list.append(df)

data_all = pd.concat(df_list)
data_all.sort_index(inplace = True)
data_all.to_csv(path_dest + tab_name + '_all.csv')

columns = list(data_all.columns)
columns.remove('OpMode')
data_all[columns] = data_all[columns].apply(pd.to_numeric)
data_all.info()


table_new/Diag_temp4.txt: 202it [00:00, 22323.06it/s]
table_new/Diag_temp3.txt: 202it [00:00, 19172.44it/s]
table_new/Diag_temp.tab: 3318it [00:23, 139.39it/s]
table_new/Diag_temp2.txt: 202it [00:00, 34876.28it/s]
table_new/Diag_temp.txt: 202it [00:00, 32061.21it/s]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3121 entries, 2021-02-25 10:49:01 to 2021-07-07 15:51:01
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   t_TTlubr  3121 non-null   float64
 1   t_TT_ab1  3121 non-null   float64
 2   t_TT_ab2  3121 non-null   float64
 3   t_Sfrnt1  3121 non-null   float64
 4   t_Sfrnt2  3121 non-null   float64
 5   t_Srear   3121 non-null   float64
 6   t_Smotor  3121 non-null   float64
 7   OpMode    3121 non-null   object 
 8   S_Speed   3121 non-null   int64  
 9   S_Load    3121 non-null   int64  
 10  tCo_H_X1  3121 non-null   int64  
 11  tCo_H_Y1  3121 non-null   float64
 12  tCo_H_Z1  3121 non-null   float64
 13  tCo_H_X2  3121 non-null   int64  
 14  tCo_H_Y2  3121 non-null   float64
 15  tCo_H_Z2  3121 non-null   int64  
 16  tCo_H_Y3  3121 non-null   int64  
 17  tCo_S2_Z  3121 non-null   int64  
dtypes: float64(10), int64(7), object(1)
memory usage: 463.3+ KB


In [30]:
#--- TABLE NEW

tabname = 'Diag_temp'
path = 'table_new/csv/'
dflist=[]
for file in os.listdir(path):
    if file.endswith('.csv') and tabname in file and  not '_all' in file:
        print(file)
        df = load_data(path + file)
        dflist.append(df)
        
data_all = pd.concat(dflist)
data_all.sort_index(inplace = True)
data_all.to_csv(path + tabname + '_all.csv')


Diag_temp.csv


In [5]:
df = pd.read_csv('TOTALE_SONDE.csv', ';')
df.set_index('DataTime', drop = True, inplace=True)
df.sort_index(inplace = True)

        
df.info()
#df.drop('OP', axis=1, inplace=True)
#fig = px.line(df)
#fig.show()

<class 'pandas.core.frame.DataFrame'>
Index: 14399 entries, 04.06.2021 08:45:01 to 14.06.2021 10:07:01
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   t_Mach    14399 non-null  float64
 1   t_HydrTa  14399 non-null  float64
 2   t_TTlubr  14399 non-null  float64
 3   t_TT_ab1  14399 non-null  float64
 4   t_TT_ab2  14399 non-null  float64
 5   t_HdStUp  14399 non-null  float64
 6   t_HdStLw  14399 non-null  float64
 7   t_Sfrnt1  14399 non-null  float64
 8   t_Sfrnt2  14399 non-null  float64
 9   t_Srear   14399 non-null  float64
 10  t_Smotor  14399 non-null  float64
 11  t_DxRiga  14399 non-null  float64
 12  t_DxFond  14399 non-null  float64
 13  t_DxAmb   14399 non-null  float64
 14  t_SxRiga  14399 non-null  float64
 15  t_SxFond  14399 non-null  float64
 16  t_SxAmb   14399 non-null  float64
 17  t_RigAnt  14399 non-null  float64
 18  t_Amb_Mn  14399 non-null  float64
 19  t_RMnPos  14399 non-null  float64
 20  t

In [5]:
import plotly.graph_objects as go
df = pd.read_csv ('TOTALE_SONDE.csv', sep=';')
df['DataTime'] = pd.to_datetime(df['DataTime'], format='%d.%m.%Y %H:%M:%S')
df.set_index('DataTime', drop = True, inplace=True)
df = df.resample('5T').mean()
fig = go.Figure()
#for column in df.keys():

visible_value = ['t_Envir', 't_Sfrnt1', 't_Sfrnt2']

for column in df.keys():
    if column in visible_value:
        visible = None
        legendgroup= 'Mario'
    else:
        visible = 'legendonly'
        legendgroup="Elettromandrino"
    fig.add_trace(go.Scatter(
                        x=df.index.tolist(), 
                        y=df[column],
                        mode='lines',
                        name=column,
                        visible=visible,
                        legendgroup=legendgroup,
                        legendgrouptitle_text = "First Group Title"
                            )
                 )
fig.show()


ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'legendgrouptitle'

Did you mean "legendgroup"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            customdata .
        dx
            Sets the

In [8]:
for (columnName, columnData) in df.iteritems():
    print('Colunm Name : ', columnName)
    #print('Column Contents : ', columnData.values)

Colunm Name :  t_Mach
Colunm Name :  t_HydrTa
Colunm Name :  t_TTlubr
Colunm Name :  t_TT_ab1
Colunm Name :  t_TT_ab2
Colunm Name :  t_HdStUp
Colunm Name :  t_HdStLw
Colunm Name :  t_Sfrnt1
Colunm Name :  t_Sfrnt2
Colunm Name :  t_Srear
Colunm Name :  t_Smotor
Colunm Name :  t_DxRiga
Colunm Name :  t_DxFond
Colunm Name :  t_DxAmb
Colunm Name :  t_SxRiga
Colunm Name :  t_SxFond
Colunm Name :  t_SxAmb
Colunm Name :  t_RigAnt
Colunm Name :  t_Amb_Mn
Colunm Name :  t_RMnPos
Colunm Name :  t_FondMn
Colunm Name :  t_ReqDom
Colunm Name :  t_ExtDom
Colunm Name :  t_ProDom
Colunm Name :  t_Envir
Colunm Name :  DataTime
